In [2]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00a 0:00:01


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
from torchvision import datasets, transforms,models
import torch
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader
import torch.nn as nn
from transformers import DefaultDataCollator
from tqdm import tqdm
import evaluate
from PIL import Image
from timeit import default_timer as timer 
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from transformers import AutoImageProcessor
import pandas as pd
import timm
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

# Preprocessing

**Mine**

In [2]:
img_dir = "/kaggle/input/classification-of-butterflies/"
train_dir = img_dir+"train_butterflies/train_split"
test_dir = img_dir +"test_butterflies"
data_transform = transforms.Compose([
    transforms.Resize(size=(256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor() 
])
train_data = datasets.ImageFolder(root = train_dir, transform=data_transform,
                                  target_transform=None)
test_data = datasets.ImageFolder(root=test_dir, 
                                 transform=data_transform,)
print(train_data.classes)
print(test_data.classes)

**Resnet**

In [4]:
img_dir = "/kaggle/input/classification-of-butterflies/"
train_dir = img_dir+"train_butterflies/train_split"
test_dir = img_dir +"test_butterflies"
data_transform = transforms.Compose([
    
    # Resize image to 224 x 224 as required by most vision models
    transforms.Resize(
        size=(224, 224)
    ),
    
    # Convert PIL image to tensor with image values in [0, 1]
    transforms.ToTensor(),
    
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
train_data = datasets.ImageFolder(root = train_dir, transform=data_transform,
                                  target_transform=None)
test_data = datasets.ImageFolder(root=test_dir, 
                                 transform=data_transform,)
print(train_data.classes)
print(test_data.classes)

['class_0', 'class_1', 'class_10', 'class_11', 'class_12', 'class_13', 'class_14', 'class_15', 'class_16', 'class_17', 'class_18', 'class_19', 'class_2', 'class_20', 'class_21', 'class_22', 'class_23', 'class_24', 'class_25', 'class_26', 'class_27', 'class_28', 'class_29', 'class_3', 'class_30', 'class_31', 'class_32', 'class_33', 'class_34', 'class_35', 'class_36', 'class_37', 'class_38', 'class_39', 'class_4', 'class_40', 'class_41', 'class_42', 'class_43', 'class_44', 'class_45', 'class_46', 'class_47', 'class_48', 'class_49', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']
['valid']


EffinsnetV2

In [8]:
img_dir = "/kaggle/input/classification-of-butterflies/"
train_dir = img_dir+"train_butterflies/train_split"
test_dir = img_dir +"test_butterflies"
data_transform = A.Compose([
                A.Resize(512,512,p=1),
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(p=0.5),
        A.CoarseDropout(p=0.5),
        ToTensorV2(p=1.0)
])
def open_img(img_path):
    img=Image.open(img_path)
    return np.array(img)
class Transform():
    def __init__(self,transform):
        self.transform=transform
    def __call__(self,image):
        return self.transform(image=image)["image"]
train_data = datasets.ImageFolder(root = train_dir, transform=Transform(data_transform),
                                  target_transform=None,loader = open_img)
test_data = datasets.ImageFolder(root=test_dir, 
                                 transform=data_transform, loader = open_img)
print(train_data.classes)
print(test_data.classes)

['class_0', 'class_1', 'class_10', 'class_11', 'class_12', 'class_13', 'class_14', 'class_15', 'class_16', 'class_17', 'class_18', 'class_19', 'class_2', 'class_20', 'class_21', 'class_22', 'class_23', 'class_24', 'class_25', 'class_26', 'class_27', 'class_28', 'class_29', 'class_3', 'class_30', 'class_31', 'class_32', 'class_33', 'class_34', 'class_35', 'class_36', 'class_37', 'class_38', 'class_39', 'class_4', 'class_40', 'class_41', 'class_42', 'class_43', 'class_44', 'class_45', 'class_46', 'class_47', 'class_48', 'class_49', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']
['valid']


**Transformer**

In [67]:
img_dir = "/kaggle/input/classification-of-butterflies/"
train_dir = img_dir+"train_butterflies/train_split"
test_dir = img_dir +"test_butterflies"
checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)
normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
data_transform = Compose([RandomResizedCrop(size), ToTensor(), normalize])
def transforms(img):
    examples = dict()
    examples["pixel_values"] = data_transform(img.convert("RGB"))
    return examples
train_data = datasets.ImageFolder(root = train_dir, transform=transforms,
                                  target_transform=None)
test_data = datasets.ImageFolder(root=test_dir, 
                                 transform=data_transform,)
print(train_data.classes)
print(test_data.classes)

['class_0', 'class_1', 'class_10', 'class_11', 'class_12', 'class_13', 'class_14', 'class_15', 'class_16', 'class_17', 'class_18', 'class_19', 'class_2', 'class_20', 'class_21', 'class_22', 'class_23', 'class_24', 'class_25', 'class_26', 'class_27', 'class_28', 'class_29', 'class_3', 'class_30', 'class_31', 'class_32', 'class_33', 'class_34', 'class_35', 'class_36', 'class_37', 'class_38', 'class_39', 'class_4', 'class_40', 'class_41', 'class_42', 'class_43', 'class_44', 'class_45', 'class_46', 'class_47', 'class_48', 'class_49', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']
['valid']


In [80]:
for i in train_data:
    print(i)
    break

(tensor([[[24, 24, 24,  ..., 28, 27, 27],
         [24, 24, 24,  ..., 28, 27, 27],
         [23, 23, 23,  ..., 27, 27, 26],
         ...,
         [33, 33, 32,  ..., 51, 42, 38],
         [34, 33, 33,  ..., 50, 40, 37],
         [34, 33, 33,  ..., 49, 39, 36]],

        [[22, 22, 22,  ..., 50, 49, 49],
         [22, 22, 22,  ..., 50, 49, 49],
         [21, 21, 21,  ..., 49, 49, 48],
         ...,
         [64, 63, 62,  ..., 55, 46, 42],
         [65, 64, 62,  ..., 53, 44, 41],
         [65, 64, 63,  ..., 52, 43, 40]],

        [[ 1,  1,  1,  ...,  2,  2,  2],
         [ 1,  1,  1,  ...,  2,  2,  2],
         [ 0,  0,  0,  ...,  1,  1,  1],
         ...,
         [ 7,  7,  6,  ..., 10,  2,  0],
         [ 8,  7,  6,  ...,  9,  2,  0],
         [ 8,  7,  6,  ...,  8,  2,  0]]], dtype=torch.uint8), 0)


# Dataloader

In [9]:
train_dataloader = DataLoader(dataset=train_data, 
                              batch_size=16, # how many samples per batch?
                              num_workers=4, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

test_dataloader = DataLoader(dataset=test_data, 
                             batch_size=16, 
                             num_workers=4, 
                             shuffle=False) # don't usually need to shuffle testing data

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7a44b2bed2a0>,
 <torch.utils.data.dataloader.DataLoader at 0x7a44b2bef6a0>)

# **Model**

In [10]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=1)

Effincnetv2

In [150]:
import gc
model = None
torch.cuda.empty_cache()
gc.collect()

0

In [11]:
class EfficientNet_V2(nn.Module):
    def __init__(self, n_out):
        super(EfficientNet_V2, self).__init__()
        # Define model
        self.effnet = timm.create_model('efficientnetv2_s', pretrained=False, num_classes=n_out)

    def forward(self, x):
        return self.effnet(x)

In [12]:
model = EfficientNet_V2(len(train_data.classes)).to(device)
from torchinfo import summary
summary(model, input_size=[1, 3, 512, 512])

Layer (type:depth-idx)                             Output Shape              Param #
EfficientNet_V2                                    [1, 50]                   --
├─EfficientNet: 1-1                                [1, 50]                   --
│    └─Conv2d: 2-1                                 [1, 24, 256, 256]         648
│    └─BatchNormAct2d: 2-2                         [1, 24, 256, 256]         48
│    │    └─Identity: 3-1                          [1, 24, 256, 256]         --
│    │    └─SiLU: 3-2                              [1, 24, 256, 256]         --
│    └─Sequential: 2-3                             [1, 256, 16, 16]          --
│    │    └─Sequential: 3-3                        [1, 24, 256, 256]         10,464
│    │    └─Sequential: 3-4                        [1, 48, 128, 128]         303,552
│    │    └─Sequential: 3-5                        [1, 64, 64, 64]           589,184
│    │    └─Sequential: 3-6                        [1, 128, 32, 32]          917,680
│    │    └─Seq

**Mine**

In [10]:
class Net(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_shape):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
            in_channels = input_dim,
            out_channels = hidden_dim,
            kernel_size = 3, stride = 1, padding = 1
            ),
            nn.ReLU(),
            nn.Conv2d(
            in_channels = hidden_dim,
            out_channels = hidden_dim,
            kernel_size = 3, stride = 1, padding = 1
            ),
            nn.ReLU(),
            nn.BatchNorm2d(hidden_dim),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
                                  )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
            in_channels = hidden_dim,
            out_channels = hidden_dim,
            kernel_size = 3, stride = 1, padding = 1
            ),
            nn.ReLU(),
            nn.Conv2d(
            in_channels = hidden_dim,
            out_channels = hidden_dim,
            kernel_size = 3, stride = 1, padding = 1
            ),
            nn.ReLU(),
            nn.BatchNorm2d(hidden_dim),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
                                  )
        self.conv3 = nn.Sequential(
            nn.Conv2d(
            in_channels = hidden_dim,
            out_channels = hidden_dim,
            kernel_size = 3, stride = 1, padding = 1
            ),
            nn.ReLU(),
            nn.Conv2d(
            in_channels = hidden_dim,
            out_channels = hidden_dim,
            kernel_size = 3, stride = 1, padding = 1
            ),
            nn.ReLU(),
            nn.BatchNorm2d(hidden_dim),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
                                  )
        self.conv4 = nn.Sequential(
            nn.Conv2d(
            in_channels = hidden_dim,
            out_channels = hidden_dim,
            kernel_size = 3, stride = 1, padding = 1
            ),
            nn.ReLU(),
            nn.Conv2d(
            in_channels = hidden_dim,
            out_channels = hidden_dim,
            kernel_size = 3, stride = 1, padding = 1
            ),
            nn.ReLU(),
            nn.BatchNorm2d(hidden_dim),
            nn.MaxPool2d(kernel_size = 2,stride = 2)
                                  )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features = hidden_dim*16*16,out_features = 2*hidden_dim),
            nn.Linear(in_features = 2*hidden_dim,out_features = hidden_dim),
            nn.Linear(in_features = hidden_dim,out_features = output_shape)
        )
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.classifier(x)
        return x

In [13]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)
model = Net(input_dim=3, # number of color channels (3 for RGB) 
                  hidden_dim=32, 
                  output_shape=len(train_data.classes)).to(device)
model

Net(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): BatchNorm2d(32, ep

In [24]:
from torchinfo import summary
summary(model, input_size=[1, 3, 256, 256]) # do a test pass through of an example input size

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [1, 50]                   --
├─Sequential: 1-1                        [1, 32, 128, 128]         --
│    └─Conv2d: 2-1                       [1, 32, 256, 256]         896
│    └─ReLU: 2-2                         [1, 32, 256, 256]         --
│    └─Conv2d: 2-3                       [1, 32, 256, 256]         9,248
│    └─ReLU: 2-4                         [1, 32, 256, 256]         --
│    └─BatchNorm2d: 2-5                  [1, 32, 256, 256]         64
│    └─MaxPool2d: 2-6                    [1, 32, 128, 128]         --
├─Sequential: 1-2                        [1, 32, 64, 64]           --
│    └─Conv2d: 2-7                       [1, 32, 128, 128]         9,248
│    └─ReLU: 2-8                         [1, 32, 128, 128]         --
│    └─Conv2d: 2-9                       [1, 32, 128, 128]         9,248
│    └─ReLU: 2-10                        [1, 32, 128, 128]         --
│    

**ResNet**

In [36]:
model = models.resnet50()
model.fc = torch.nn.Sequential(
    torch.nn.Linear(
        in_features=2048,
        out_features=len(train_data.classes)
    ),
)
from torchinfo import summary
summary(model, input_size=[1, 3, 256, 256])

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 50]                   --
├─Conv2d: 1-1                            [1, 64, 128, 128]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 128, 128]         128
├─ReLU: 1-3                              [1, 64, 128, 128]         --
├─MaxPool2d: 1-4                         [1, 64, 64, 64]           --
├─Sequential: 1-5                        [1, 256, 64, 64]          --
│    └─Bottleneck: 2-1                   [1, 256, 64, 64]          --
│    │    └─Conv2d: 3-1                  [1, 64, 64, 64]           4,096
│    │    └─BatchNorm2d: 3-2             [1, 64, 64, 64]           128
│    │    └─ReLU: 3-3                    [1, 64, 64, 64]           --
│    │    └─Conv2d: 3-4                  [1, 64, 64, 64]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 64, 64]           128
│    │    └─ReLU: 3-6                    [1, 64, 64, 64]           --
│ 

**Transformer**

In [74]:
labels = train_data.classes
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
print(label2id)

{'class_0': '0', 'class_1': '1', 'class_10': '2', 'class_11': '3', 'class_12': '4', 'class_13': '5', 'class_14': '6', 'class_15': '7', 'class_16': '8', 'class_17': '9', 'class_18': '10', 'class_19': '11', 'class_2': '12', 'class_20': '13', 'class_21': '14', 'class_22': '15', 'class_23': '16', 'class_24': '17', 'class_25': '18', 'class_26': '19', 'class_27': '20', 'class_28': '21', 'class_29': '22', 'class_3': '23', 'class_30': '24', 'class_31': '25', 'class_32': '26', 'class_33': '27', 'class_34': '28', 'class_35': '29', 'class_36': '30', 'class_37': '31', 'class_38': '32', 'class_39': '33', 'class_4': '34', 'class_40': '35', 'class_41': '36', 'class_42': '37', 'class_43': '38', 'class_44': '39', 'class_45': '40', 'class_46': '41', 'class_47': '42', 'class_48': '43', 'class_49': '44', 'class_5': '45', 'class_6': '46', 'class_7': '47', 'class_8': '48', 'class_9': '49'}


In [ ]:


accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

data_collator = DefaultDataCollator(return_tensors="pt")
model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)
training_args = TrainingArguments(
    output_dir="my_awesome_food_model",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=None,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)
trainer.train()

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ······


wandb: ERROR API key must be 40 characters long, yours was 6
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

# **Training**

In [15]:
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer):
    # Put model in train mode
    model.train()
    
    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0
    train 
    # Loop through data loader data batches
    for (X, y) in tqdm(dataloader):
        # Send data to target device
        X, y = X.to(device).float(), y.to(device).long()
        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        #print(y_pred.size(),y.size())
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

In [16]:
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):
    
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in range(epochs):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        #test_loss, test_acc = test_step(model=model,
        #    dataloader=test_dataloader,
        #    loss_fn=loss_fn)
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
        #    f"test_loss: {test_loss:.4f} | "
         #   f"test_acc: {test_acc:.4f}"
        )
        if len(results['train_acc'])==0 or results['train_acc'][-1]<train_acc:
            EPOCH = epoch
            LOSS = train_loss
            PATH = "model.pt"

            torch.save({
            'epoch': EPOCH,
            'model_state_dict': model,
            'optimizer_state_dict': optimizer,
            'loss': LOSS,
            }, PATH)
            print('saving model')
        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        #results["test_loss"].append(test_loss)
        #results["test_acc"].append(test_acc)

    # 6. Return the filled results at the end of the epochs
    return results

In [17]:
torch.manual_seed(42) 
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 40

# Recreate an instance of Net
'''model = Net(input_dim=3, # number of color channels (3 for RGB) 
                  hidden_dim=32, 
                  output_shape=len(train_data.classes)).to(device)'''

# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [20]:
model.train()
# Start the timer
start_time = timer()

# Train model_0 
model_0_results = train(model=model, 
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn, 
                        epochs=5)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

100%|██████████| 310/310 [04:06<00:00,  1.26it/s]


Epoch: 1 | train_loss: 0.3034 | train_acc: 0.9025 | 
saving model


100%|██████████| 310/310 [04:05<00:00,  1.26it/s]


Epoch: 2 | train_loss: 0.3127 | train_acc: 0.8976 | 


  0%|          | 0/310 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [42]:
# Start the timer
start_time = timer()

# Train model_0 
model_0_results = train(model=model, 
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn, 
                        epochs=10)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

100%|██████████| 155/155 [00:20<00:00,  7.47it/s]


Epoch: 1 | train_loss: 1.3877 | train_acc: 0.6064 | 


100%|██████████| 155/155 [00:20<00:00,  7.46it/s]


Epoch: 2 | train_loss: 0.9609 | train_acc: 0.7195 | 


100%|██████████| 155/155 [00:20<00:00,  7.45it/s]


...

Epoch: 8 | train_loss: 0.1891 | train_acc: 0.9447 | 


100%|██████████| 155/155 [00:21<00:00,  7.32it/s]


Epoch: 9 | train_loss: 0.1346 | train_acc: 0.9588 | 


100%|██████████| 155/155 [00:21<00:00,  7.31it/s]

Epoch: 10 | train_loss: 0.1786 | train_acc: 0.9432 | 
Total training time: 209.778 seconds


**Loading + Test**

In [21]:
EPOCH = 20
LOSS = 0.2692
PATH = "model.pt"

torch.save({
            'epoch': EPOCH,
            'model_state_dict': model,
            'optimizer_state_dict': optimizer,
            'loss': LOSS,
            }, PATH)

In [22]:
PATH = "/kaggle/working/model.pt"
checkpoint = torch.load(PATH)
model = checkpoint['model_state_dict']
optimizer = checkpoint['optimizer_state_dict']
epoch = checkpoint['epoch']
loss = checkpoint['loss']
model.to(device)
model.eval()

EfficientNet_V2(
  (effnet): EfficientNet(
    (conv_stem): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (drop_path): Identity()
        )
        (1): ConvBnAct(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNormAct2d(
            24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace

In [23]:
from PIL import Image
import os,cv2
test = {
    'imgs': [],
    'index': []
}

test_path = '/kaggle/input/classification-of-butterflies/test_butterflies/valid'
for file in os.listdir(test_path):
    img = cv2.imread(f'{test_path}/{file}')
    #img = Image.fromarray(img, 'RGB')
    test['imgs'].append(Transform(data_transform).__call__(img))
    test['index'].append(int(file.replace('.jpg', '')))

test = {
    'imgs': np.array(test['imgs']),
    'index': np.array(test['index'])
}
test

{'imgs': array([[[[112, 112, 110, ..., 130, 129, 128],
          [111, 112, 110, ..., 130, 130, 129],
          [109, 111, 108, ..., 130, 130, 130],
          ...,
          [ 54,  56,  58, ...,  79,  76,  69],
          [ 54,  57,  60, ...,  71,  64,  62],
          [ 55,  58,  61, ...,  65,  62,  63]],
 
         [[149, 149, 147, ..., 166, 166, 165],
          [147, 149, 147, ..., 166, 166, 165],
          [144, 146, 144, ..., 166, 166, 166],
          ...,
          [ 84,  85,  87, ..., 204, 199, 188],
          [ 85,  87,  90, ..., 193, 184, 181],
          [ 88,  90,  93, ..., 185, 182, 184]],
 
         [[163, 163, 161, ..., 182, 182, 180],
          [162, 163, 161, ..., 182, 182, 181],
          [160, 161, 158, ..., 182, 182, 182],
          ...,
          [ 99, 100, 102, ..., 173, 169, 159],
          [ 99, 102, 104, ..., 163, 155, 152],
          [101, 103, 106, ..., 156, 153, 155]]],
 
 
        [[[ 10,   9,   8, ..., 154, 144, 127],
          [  9,   8,   7, ..., 154, 144, 1

In [26]:
preds = torch.empty(0).long().to(device)
for i in test['imgs']:
    i = torch.from_numpy(i)
    y_pred = model(i.unsqueeze_(0).to(device).float())
    y_pred_class = torch.argmax(torch.softmax(y_pred,dim=1),dim=1)
    preds = torch.cat((preds,y_pred_class),0)
preds

tensor([47, 12, 46, 46, 47,  0, 47, 40, 34, 47, 20,  0, 46, 27, 40, 26, 34, 36,
         0, 22, 27, 15, 27, 23, 32, 23,  0, 47, 34,  0,  0,  0, 40,  0, 36,  0,
        16,  0,  7,  4,  0, 42, 17, 15, 40, 19,  0, 40, 28, 29,  7, 15, 26,  0,
        40, 47, 32, 17,  0,  0, 33, 43,  0,  0, 20, 34, 22, 43, 12, 39,  7,  4,
         6,  2,  7, 36,  2, 22, 39, 15,  0, 30, 40, 34, 40,  0,  0, 48,  0, 26,
        40, 46, 22,  0, 49, 34,  0, 23,  7,  0, 36, 40,  0, 10, 28, 13,  0, 40,
         0,  1, 49, 46,  0, 27, 47, 27, 44,  0,  0, 20, 40, 47, 40, 40, 42, 29,
        14,  0, 36, 27, 14, 23, 15, 48,  2,  0,  0,  0,  0, 24,  0, 47,  0, 26,
        16, 25, 43, 10, 15, 46, 27, 43, 40, 40,  0, 40, 27, 47, 40, 31, 27, 15,
        22, 27, 40, 15, 40, 11, 26, 47,  0, 40,  0, 27, 14, 14, 49, 40, 40, 27,
        30, 27,  0, 23, 48,  2,  9, 22, 34, 44,  0, 36,  7, 36,  2,  0, 14,  0,
         0,  0, 44, 40,  0, 27, 34, 20, 15, 22, 14, 46, 15, 31,  0,  0, 22, 13,
         0,  7,  9, 37,  2,  0,  7, 10, 

In [25]:
preds = torch.empty(0).long().to(device)
for X,y in test_dataloader:
    y_pred = model(X.to(device))
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    preds = torch.cat((preds,y_pred_class),0)
    #print(preds)

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.10/site-packages/torchvision/datasets/folder.py", line 231, in __getitem__
    sample = self.transform(sample)
  File "/opt/conda/lib/python3.10/site-packages/albumentations/core/composition.py", line 193, in __call__
    raise KeyError("You have to pass data to augmentations as named arguments, for example: aug(image=image)")
KeyError: 'You have to pass data to augmentations as named 

In [27]:
class_names = train_data.classes
class_count = len(class_names)
class_count

50

In [28]:
y_preds = np.array(preds.to('cpu'))
pred_classes = []
for i in y_preds:
    pred_classes.append(class_names[i])
#print(pred_classes)
df = pd.DataFrame(pred_classes, columns=['label'])
df['label'] = df['label'].str.replace(r"class_",'')
df['label'] = df['label'].astype(int)
df = df.reset_index()
df['index'] = test['index']
df

,index,label
0,208,7
1,45,2
2,56,6
3,89,6
4,20,7
...,...,...
245,213,45
246,136,10
247,90,4
248,25,33


In [29]:
df.to_csv('predictions.csv', index=False)